# Monitors Debugger Notebook - Verify Monitors Setup and Execution (Optionally)

In [1]:
%%time

from datetime import datetime, timedelta, timezone
import json
import os
import re
import boto3
from time import sleep
from threading import Thread

import io
import random
import pandas as pd

from sagemaker import get_execution_role, session, Session, image_uris
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.processing import ProcessingJob
from sagemaker.serializers import CSVSerializer

from sagemaker.model import Model
from sagemaker.model_monitor import (
    BiasAnalysisConfig,
    CronExpressionGenerator,
    DataCaptureConfig,
    EndpointInput,
    ExplainabilityAnalysisConfig,
    ModelBiasMonitor,
    ModelExplainabilityMonitor,
)

from sagemaker.clarify import (
    BiasConfig,
    DataConfig,
    ModelConfig,
    ModelPredictedLabelConfig,
    SHAPConfig,
)

from threading import Timer

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
CPU times: user 1.23 s, sys: 142 ms, total: 1.38 s
Wall time: 1.85 s


In [2]:
sagemaker_session = Session()
sagemaker_client = sagemaker_session.sagemaker_client
sagemaker_runtime_client = sagemaker_session.sagemaker_runtime_client

# Retrieve the default Amazon S3 bucket associated with the SageMaker session.
bucket = sagemaker_session.default_bucket()
print("Bucket:", bucket)

# Get the IAM role associated with the current SageMaker notebook or environment.
role = get_execution_role()
print("RoleArn:", role)

# Get the AWS region name for the current session.
region = boto3.Session().region_name
print("Region", region)

# Retrieve the AWS account ID of the caller using the Security Token Service (STS) client.
account_id = boto3.client("sts").get_caller_identity().get("Account")

# Create a Boto3 client for the SageMaker service, specifying the AWS region.
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

# Create an S3 client
s3 = boto3.client('s3')

Bucket: sagemaker-us-east-1-084375567266
RoleArn: arn:aws:iam::084375567266:role/service-role/SageMaker-ExecutionRole-20250222T162355
Region us-east-1


In [12]:
!aws sagemaker list-monitoring-schedules --endpoint-name "retain-ai-xgb-endpoint"

{
    "MonitoringScheduleSummaries": [
        {
            "MonitoringScheduleName": "monitoring-schedule-2025-02-25-03-03-47-297",
            "MonitoringScheduleArn": "arn:aws:sagemaker:us-east-1:084375567266:monitoring-schedule/monitoring-schedule-2025-02-25-03-03-47-297",
            "CreationTime": "2025-02-25T03:03:47.968000+00:00",
            "LastModifiedTime": "2025-02-25T04:19:03.650000+00:00",
            "MonitoringScheduleStatus": "Scheduled",
            "EndpointName": "retain-ai-xgb-endpoint",
            "MonitoringJobDefinitionName": "model-bias-job-definition-2025-02-25-03-03-47-297",
            "MonitoringType": "ModelBias"
        }
    ]
}


In [13]:
# !aws sagemaker delete-monitoring-schedule --monitoring-schedule-name "monitoring-schedule-2025-02-25-03-03-47-297"

In [6]:
!aws sagemaker list-monitoring-executions --monitoring-schedule-name  "monitoring-schedule-2025-02-24-05-33-49-009"

{
    "MonitoringExecutionSummaries": [
        {
            "MonitoringScheduleName": "monitoring-schedule-2025-02-24-05-33-49-009",
            "ScheduledTime": "2025-02-25T02:00:00+00:00",
            "CreationTime": "2025-02-25T02:01:13.349000+00:00",
            "LastModifiedTime": "2025-02-25T02:03:49.232000+00:00",
            "MonitoringExecutionStatus": "Failed",
            "EndpointName": "retain-ai-xgb-endpoint",
            "FailureReason": "No S3 objects found under S3 URL \"s3://sagemaker-us-east-1-084375567266/aai-540-group-3-final-project/ground_truth_data/2025-02-24-05/2025/02/25/01\" given in input data source. Please ensure that the bucket exists in the selected region (us-east-1), that objects exist under that S3 prefix, and that the role \"arn:aws:iam::084375567266:role/service-role/SageMaker-ExecutionRole-20250222T162355\" has \"s3:ListBucket\" permissions on bucket \"sagemaker-us-east-1-084375567266\".",
            "MonitoringJobDefinitionName": "model-bias-jo

In [23]:
!aws s3 ls s3://sagemaker-us-east-1-084375567266/aai-540-group-3-final-project/ground_truth_data/ --recursive
aai-540-group-3-final-project/ground_truth_data/2025-02-24-05
aai-540-group-3-final-project/ground_truth_data/2025-02-24-05/2025/02/24/04/3331.jsonl

2025-02-24 05:33:32       2274 aai-540-group-3-final-project/ground_truth_data/2025-02-24-05/2025/02/24/04/3331.jsonl
2025-02-24 05:34:32      11388 aai-540-group-3-final-project/ground_truth_data/2025-02-24-05/2025/02/24/05/3431.jsonl
2025-02-24 05:35:32      13660 aai-540-group-3-final-project/ground_truth_data/2025-02-24-05/2025/02/24/05/3531.jsonl
2025-02-24 05:36:33      13660 aai-540-group-3-final-project/ground_truth_data/2025-02-24-05/2025/02/24/05/3631.jsonl
2025-02-24 05:37:33      13657 aai-540-group-3-final-project/ground_truth_data/2025-02-24-05/2025/02/24/05/3732.jsonl
2025-02-24 05:38:33      13662 aai-540-group-3-final-project/ground_truth_data/2025-02-24-05/2025/02/24/05/3832.jsonl
2025-02-24 05:39:33      13661 aai-540-group-3-final-project/ground_truth_data/2025-02-24-05/2025/02/24/05/3932.jsonl
2025-02-24 05:40:33      11380 aai-540-group-3-final-project/ground_truth_data/2025-02-24-05/2025/02/24/05/4032.jsonl
2025-02-24 05:41:33      13657 aai-540-group-3-final-pro

In [11]:
!aws sagemaker describe-processing-job --processing-job-name groundtruth-merge-202502240600-e05cd0ffcd383fc89a70e8bf

{
    "ProcessingInputs": [
        {
            "InputName": "groundtruth_input_1",
            "AppManaged": false,
            "S3Input": {
                "S3Uri": "s3://sagemaker-us-east-1-084375567266/aai-540-group-3-final-project/ground_truth_data/2025-02-24-05/2025/02/24/05",
                "LocalPath": "/opt/ml/processing/groundtruth/2025/02/24/05",
                "S3DataType": "S3Prefix",
                "S3InputMode": "File",
                "S3DataDistributionType": "FullyReplicated",
                "S3CompressionType": "None"
            }
        },
        {
            "InputName": "endpoint_input_1",
            "AppManaged": false,
            "S3Input": {
                "S3Uri": "s3://sagemaker-us-east-1-084375567266/aai-540-group-3-final-project/data/monitoring/retain-ai-xgb-endpoint/AllTraffic/2025/02/24/05",
                "LocalPath": "/opt/ml/processing/input/endpoint/retain-ai-xgb-endpoint/AllTraffic/2025/02/24/05",
                "S3DataType": "S3Prefix

In [8]:
!aws s3 ls s3://sagemaker-us-east-1-084375567266/aai-540-group-3-final-project/data/monitoring --recursive

2025-02-24 03:58:53      25302 aai-540-group-3-final-project/data/monitoring/retain-ai-xgb-endpoint/AllTraffic/2025/02/24/03/57-42-336-70986326-735f-4a47-8602-dd3231b1fbb4.jsonl
2025-02-24 04:00:53      42189 aai-540-group-3-final-project/data/monitoring/retain-ai-xgb-endpoint/AllTraffic/2025/02/24/03/59-43-813-12a87154-813d-4068-ace0-031931b14b48.jsonl
2025-02-24 04:07:23      50611 aai-540-group-3-final-project/data/monitoring/retain-ai-xgb-endpoint/AllTraffic/2025/02/24/04/06-16-746-09429515-3bc0-4195-b04d-f3a2030d5263.jsonl
2025-02-24 04:08:23      50616 aai-540-group-3-final-project/data/monitoring/retain-ai-xgb-endpoint/AllTraffic/2025/02/24/04/07-18-811-9fe6d466-52c3-40eb-817c-ef7a51830c10.jsonl
2025-02-24 04:09:23      50611 aai-540-group-3-final-project/data/monitoring/retain-ai-xgb-endpoint/AllTraffic/2025/02/24/04/08-20-741-f42ecccf-bd18-4ed5-8fb1-c53f48c5a069.jsonl
2025-02-24 04:10:33      50604 aai-540-group-3-final-project/data/monitoring/retain-ai-xgb-endpoint/AllTraffic

In [ ]:
print("Waiting for captures to show up", end="")

    capture_files = sorted(S3Downloader.list(f"{s3_capture_upload_path}/{endpoint_name}"))
    if capture_files:
        capture_file = S3Downloader.read_file(capture_files[-1]).split("\n")
        capture_record = json.loads(capture_file[0])
        if "inferenceId" in capture_record["eventMetadata"]:
            break
    print(".", end="", flush=True)
    sleep(1)
print()
print("Found Capture Files:")
print("\n ".join(capture_files[-3:]))